In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

print("🚀 Starting Credit Risk Scoring Engine (Fast Version)...")

# ============================================================================
# STEP 1: LOAD FINAL FEATURES
# ============================================================================
print("\n📂 Loading final features...")

df = pd.read_csv('DATA/processed/customer_final_features.csv')

print(f"✅ Loaded {len(df):,} customers")
print(f"   Features: {df.shape[1]} columns")

# ============================================================================
# STEP 2: HARD REJECT RULES (Vectorized)
# ============================================================================
print("\n🚫 Applying hard reject rules...")

# Rule 1: 90+ day default
reject_1 = df['dpd_90_count_6m'] > 0

# Rule 2: Multiple recent 30+ day defaults
reject_2 = df['dpd_30_count_3m'] > 1

# Rule 3: Employer mismatch + low liquidity
reject_3 = (df['salary_creditor_consistent'] == 0) & (df['liquidity_flag'] == 'LOW')

# Rule 4: Missing salary + poor discipline
reject_4 = (df['salary_missing_months'] > 2) & (df['payment_discipline_flag'] == 'POOR')

# Combine all hard rejects
df['hard_reject'] = reject_1 | reject_2 | reject_3 | reject_4

# Reason (simplified for speed)
df['hard_reject_reason'] = np.where(
    df['hard_reject'],
    'Critical risk factors detected',
    None
)

print(f"   Hard rejects: {df['hard_reject'].sum():,} ({df['hard_reject'].mean()*100:.2f}%)")

# ============================================================================
# STEP 3: CALCULATE RISK SCORE (Vectorized)
# ============================================================================
print("\n🧮 Calculating risk scores...")

# Start with base score
score = pd.Series(100, index=df.index)

# ========== BUREAU RISK (40 points) ==========
score -= df['dpd_30_count_6m'] * 10
score -= df['dpd_15_count_6m'] * 3

score -= np.where(df['bureau_risk_flag'] == 'HIGH', 20, 0)
score -= np.where(df['bureau_risk_flag'] == 'MEDIUM', 10, 0)

# ========== PAYMENT DISCIPLINE (25 points) ==========
score -= df['total_late_30_6m'] * 8
score -= df['total_late_90_6m'] * 15

score -= np.where(df['payment_discipline_flag'] == 'POOR', 15, 0)
score -= np.where(df['payment_discipline_flag'] == 'MODERATE', 7, 0)

score -= df['recent_payment_stress'] * 10

# ========== SALARY STABILITY (20 points) ==========
score -= np.where(df['salary_amount_cv'] > 0.20, 10, 
          np.where(df['salary_amount_cv'] > 0.15, 5, 0))

score -= np.where(df['salary_date_std'] > 7, 8,
          np.where(df['salary_date_std'] > 5, 4, 0))

score -= np.where(df['salary_creditor_consistent'] == 0, 15, 0)
score -= df['salary_missing_months'] * 5

# ========== LIQUIDITY (15 points) ==========
score -= np.where(df['liquidity_flag'] == 'LOW', 15, 0)
score -= np.where(df['liquidity_flag'] == 'MODERATE', 7, 0)

# Balance coverage
coverage = df['avg_monthly_balance_6m'] / df['total_emi_monthly'].replace(0, 1)
score -= np.where(coverage < 1.0, 10, np.where(coverage < 1.5, 5, 0))

score -= df['inward_bounce_count_3m'] * 10

# Floor and ceiling
score = score.clip(0, 100)

# Set hard rejects to 0
df['risk_score'] = np.where(df['hard_reject'], 0, score)

print(f"   Score range: {df['risk_score'].min():.0f} to {df['risk_score'].max():.0f}")
print(f"   Average score: {df['risk_score'].mean():.1f}")

# ============================================================================
# STEP 4: MAKE LOAN DECISION (Vectorized)
# ============================================================================
print("\n⚖️ Making loan decisions...")

# Initialize
df['loan_decision'] = 'REVIEW'
df['decision_reason'] = 'Borderline metrics'

# Hard rejects
df.loc[df['hard_reject'], 'loan_decision'] = 'REJECT'
df.loc[df['hard_reject'], 'decision_reason'] = df.loc[df['hard_reject'], 'hard_reject_reason']

# Score-based decisions (for non-hard-rejects)
mask_not_rejected = ~df['hard_reject']

# Approvals (score >= 75)
approve_mask = mask_not_rejected & (df['risk_score'] >= 75)
df.loc[approve_mask, 'loan_decision'] = 'APPROVE'
df.loc[approve_mask, 'decision_reason'] = 'Strong profile (score: ' + df.loc[approve_mask, 'risk_score'].astype(int).astype(str) + ')'

# Rejections (score < 55)
reject_mask = mask_not_rejected & (df['risk_score'] < 55)
df.loc[reject_mask, 'loan_decision'] = 'REJECT'
df.loc[reject_mask, 'decision_reason'] = 'Risk too high (score: ' + df.loc[reject_mask, 'risk_score'].astype(int).astype(str) + ')'

# Review stays as default for 55-74

# ============================================================================
# STEP 5: DECISION STATISTICS
# ============================================================================
print("\n" + "="*60)
print("✅ CREDIT RISK ENGINE COMPLETE!")
print("="*60)

print(f"\n📊 Decision Distribution:")
for decision, count in df['loan_decision'].value_counts().items():
    pct = (count / len(df)) * 100
    print(f"   {decision:8s}: {count:7,} ({pct:5.1f}%)")

print(f"\n📈 Score Distribution by Decision:")
for decision in ['APPROVE', 'REVIEW', 'REJECT']:
    subset = df[df['loan_decision'] == decision]['risk_score']
    if len(subset) > 0:
        print(f"   {decision:8s}: avg={subset.mean():.1f}, min={subset.min():.0f}, max={subset.max():.0f}")

# ============================================================================
# STEP 6: VALIDATION AGAINST ACTUAL DEFAULTS
# ============================================================================
print("\n🔍 Validating against actual defaults...")

if 'TARGET' in df.columns:
    approved = df[df['loan_decision'] == 'APPROVE']
    rejected = df[df['loan_decision'] == 'REJECT']
    
    if len(approved) > 0:
        print(f"\n   Approved default rate: {approved['TARGET'].mean()*100:.2f}%")
    
    if len(rejected) > 0:
        print(f"   Rejected default rate: {rejected['TARGET'].mean()*100:.2f}%")
    
    if len(approved) > 0 and len(rejected) > 0:
        improvement = (rejected['TARGET'].mean() - approved['TARGET'].mean()) / rejected['TARGET'].mean() * 100
        print(f"\n   ✅ Engine reduced default risk by {improvement:.1f}%")

# ============================================================================
# STEP 7: SAMPLE DECISIONS
# ============================================================================
print("\n📋 Sample Decisions:")

sample_cols = ['customer_id', 'risk_score', 'bureau_risk_flag', 'payment_discipline_flag', 'loan_decision']

print("\nAPPROVED (Sample):")
print(df[df['loan_decision'] == 'APPROVE'][sample_cols].head(3).to_string(index=False))

print("\n\nREVIEW (Sample):")
print(df[df['loan_decision'] == 'REVIEW'][sample_cols].head(3).to_string(index=False))

print("\n\nREJECTED (Sample):")
print(df[df['loan_decision'] == 'REJECT'][sample_cols].head(3).to_string(index=False))

# ============================================================================
# STEP 8: SAVE FINAL OUTPUT
# ============================================================================
output_file = Path('DATA/processed/customer_credit_decisions.csv')
df.to_csv(output_file, index=False)

print(f"\n💾 Saved: {output_file}")
print(f"   Columns: {df.shape[1]}")
print(f"   Rows: {len(df):,}")

print("\n" + "="*60)
print("🎉 CREDIT ANALYSIS ENGINE COMPLETE!")
print("="*60)
print("\n✅ Your LSP Credit Risk Engine is Ready!")
print("✅ Next: Dashboard or ML Model")

🚀 Starting Credit Risk Scoring Engine (Fast Version)...

📂 Loading final features...
✅ Loaded 307,511 customers
   Features: 48 columns

🚫 Applying hard reject rules...
   Hard rejects: 1,324 (0.43%)

🧮 Calculating risk scores...
   Score range: 0 to 100
   Average score: 76.5

⚖️ Making loan decisions...

✅ CREDIT RISK ENGINE COMPLETE!

📊 Decision Distribution:
   APPROVE : 295,065 ( 96.0%)
   REVIEW  :   9,495 (  3.1%)
   REJECT  :   2,951 (  1.0%)

📈 Score Distribution by Decision:
   APPROVE : avg=77.4, min=75, max=100
   REVIEW  : avg=69.2, min=56, max=74
   REJECT  : avg=17.1, min=0, max=54

🔍 Validating against actual defaults...

   Approved default rate: 7.90%
   Rejected default rate: 14.00%

   ✅ Engine reduced default risk by 43.6%

📋 Sample Decisions:

APPROVED (Sample):
 customer_id  risk_score bureau_risk_flag payment_discipline_flag loan_decision
      100002        75.0              LOW                    GOOD       APPROVE
      100003        75.0              LOW    

In [3]:
import pandas as pd

df = pd.read_csv('DATA/processed/customer_credit_decisions.csv')

print(f"Current columns: {df.shape[1]}")
print("\nAll columns:")
for i, col in enumerate(df.columns, 1):
    print(f"{i:2d}. {col}")

Current columns: 53

All columns:
 1. customer_id
 2. TARGET
 3. AMT_INCOME_TOTAL
 4. AMT_ANNUITY
 5. active_loans_count
 6. dpd_15_count_6m
 7. dpd_30_count_6m
 8. dpd_90_count_6m
 9. max_dpd_6m
10. dpd_30_count_3m
11. hard_reject_flag
12. bureau_risk_flag
13. total_payments_6m
14. total_late_15_6m
15. total_late_30_6m
16. total_late_60_6m
17. total_late_90_6m
18. max_days_late_6m
19. avg_days_late_6m
20. total_late_30_3m
21. total_late_90_3m
22. payment_discipline_flag
23. recent_payment_stress
24. avg_balance_cc
25. total_drawings_cc
26. avg_credit_limit
27. max_utilization
28. total_payments_cc
29. dpd_count_cc
30. avg_balance_pos
31. dpd_count_pos
32. total_credit_activity
33. total_dpd_count
34. avg_monthly_balance_6m
35. total_emi_monthly
36. net_cash_surplus_6m
37. total_credit_6m
38. total_debit_6m
39. liquidity_flag
40. cashflow_health
41. inward_bounce_count_3m
42. avg_salary_6m
43. salary_txn_count_6m
44. salary_amount_cv
45. salary_date_std
46. salary_creditor_consistent
4

In [4]:
import pandas as pd
from pathlib import Path

print("🧹 Cleaning dataset to final 26 columns...")

# Load current dataset
df = pd.read_csv('DATA/processed/customer_credit_decisions.csv')

print(f"Current shape: {df.shape}")

# ============================================================================
# DEFINE EXACT 26 COLUMNS NEEDED
# ============================================================================

required_columns = [
    # 1. Customer Identity (1)
    'customer_id',
    
    # 2. Bureau / Credit History (7)
    'bureau_score',  # Needs to be derived if missing
    'dpd_15_count_6m',
    'dpd_30_count_6m',
    'dpd_90_count_6m',
    'dpd_30_count_3m',
    'active_loans_count',
    'total_emi_monthly',
    
    # 3. Cash-Flow & Liquidity (5)
    'total_credit_6m',
    'total_debit_6m',
    'avg_monthly_balance_6m',
    'net_cash_surplus_6m',
    'inward_bounce_count_3m',
    
    # 4. Salary Integrity (6)
    'avg_salary_6m',
    'salary_txn_count_6m',
    'salary_amount_cv',
    'salary_date_std',
    'salary_creditor_consistent',
    'salary_missing_months',
    
    # 5. Derived Risk Flags (4)
    'salary_stability_flag',
    'liquidity_flag',
    'bureau_risk_flag',
    'hard_reject_flag',
    
    # 6. Final Outputs (3)
    'risk_score',
    'loan_decision',
    'decision_reason'
]

# ============================================================================
# CREATE BUREAU_SCORE IF MISSING
# ============================================================================

if 'bureau_score' not in df.columns:
    print("\n📊 Creating bureau_score from risk signals...")
    
    # Simple bureau score proxy (300-900 scale like CIBIL)
    base_score = 750
    
    score = pd.Series(base_score, index=df.index)
    score -= df['dpd_90_count_6m'] * 100
    score -= df['dpd_30_count_6m'] * 50
    score -= df['dpd_15_count_6m'] * 20
    
    score = score.clip(300, 900)
    df['bureau_score'] = score.astype(int)

# ============================================================================
# VALIDATE ALL REQUIRED COLUMNS EXIST
# ============================================================================

missing_cols = [col for col in required_columns if col not in df.columns]

if missing_cols:
    print(f"\n⚠️ Missing columns: {missing_cols}")
    print("Creating placeholder columns...")
    
    for col in missing_cols:
        if col == 'total_emi_monthly':
            df[col] = df.get('AMT_ANNUITY', 0)
        else:
            df[col] = 0

# ============================================================================
# SELECT ONLY REQUIRED COLUMNS
# ============================================================================

final_df = df[required_columns].copy()

print(f"\n✅ Final dataset created")
print(f"   Shape: {final_df.shape}")
print(f"   Columns: {len(final_df.columns)}")

# ============================================================================
# SAVE CLEAN DATASET
# ============================================================================

output_file = Path('DATA/processed/credit_analysis_final.csv')
final_df.to_csv(output_file, index=False)

print(f"\n💾 Saved clean dataset: {output_file}")

print("\n📋 Final Column List:")
for i, col in enumerate(final_df.columns, 1):
    print(f"   {i:2d}. {col}")

print("\n" + "="*60)
print("✅ DATASET CLEANED TO EXACTLY 26 COLUMNS")
print("="*60)

🧹 Cleaning dataset to final 26 columns...
Current shape: (307511, 53)

📊 Creating bureau_score from risk signals...

✅ Final dataset created
   Shape: (307511, 26)
   Columns: 26

💾 Saved clean dataset: DATA\processed\credit_analysis_final.csv

📋 Final Column List:
    1. customer_id
    2. bureau_score
    3. dpd_15_count_6m
    4. dpd_30_count_6m
    5. dpd_90_count_6m
    6. dpd_30_count_3m
    7. active_loans_count
    8. total_emi_monthly
    9. total_credit_6m
   10. total_debit_6m
   11. avg_monthly_balance_6m
   12. net_cash_surplus_6m
   13. inward_bounce_count_3m
   14. avg_salary_6m
   15. salary_txn_count_6m
   16. salary_amount_cv
   17. salary_date_std
   18. salary_creditor_consistent
   19. salary_missing_months
   20. salary_stability_flag
   21. liquidity_flag
   22. bureau_risk_flag
   23. hard_reject_flag
   24. risk_score
   25. loan_decision
   26. decision_reason

✅ DATASET CLEANED TO EXACTLY 26 COLUMNS
